In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path

import numpy as np
import pandas as pd

from typing import Union
from tqdm.notebook import tqdm

In [3]:
path = Path('/kaggle/m5_forecasting/')
assert(path.exists())

In [4]:
!ls {path}

calendar.csv
m5_best_1.pth
m5_best_2.pth
m5_dt
m5-forecasting-accuracy-publicleaderboard.csv
m5-forecasting-accuracy-publicleaderboard-rank.csv
m5_model
m5_model_0_export.pkl
m5_model_0.lgb
m5_model_1000.lgb
m5_model_1200.lgb
m5_model_1.lgb
m5_model_2_export.pkl
m5_model_2.lgb
m5_model_3_export.pkl
m5_model_3.lgb
m5_model_4.lgb
m5_model_7_export.pkl
m5_model_800.lgb
m5_model_8_export.pkl
m5_model_9_export.pkl
m5_model_best.lgb
m5_model_best.lgb.tgz
m5_model_binary.lgb
m5_model.lgb
m5_model.pth
m5_models.tgz
m5_model_tweedie_1.2_800.lgb
models
sales_train_evaluation.csv
sales_train_validation.csv
sample_submission.csv
sell_prices.csv
temperature_us.zip
unemployment_us.csv
walmartTrends0.csv
weather


In [5]:
## new train data
df_train_full = pd.read_csv(path/"sales_train_evaluation.csv")
df_train_full.iloc[:, -31:].head()

,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0,1,1,0,0,0,2,0,3,5,...,2,4,0,0,0,0,3,3,0,1
1,0,0,0,0,1,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,1,1,1,0,0,1,1,0,2,1,...,1,0,2,0,0,0,2,3,0,1
3,3,7,2,0,0,1,2,4,1,6,...,1,1,0,4,0,1,3,0,2,6
4,2,2,4,1,0,2,3,1,0,3,...,0,0,0,2,1,0,0,2,1,0


In [6]:
## evaluation metric
## from https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834 and edited to get scores at all levels
class WRMSSEEvaluator(object):

    def __init__(self, train_df: pd.DataFrame, valid_df: pd.DataFrame, calendar: pd.DataFrame, prices: pd.DataFrame):
        train_y = train_df.loc[:, train_df.columns.str.startswith('d_')]
        train_target_columns = train_y.columns.tolist()
        weight_columns = train_y.iloc[:, -28:].columns.tolist()

        train_df['all_id'] = 0  # for lv1 aggregation

        id_columns = train_df.loc[:, ~train_df.columns.str.startswith('d_')].columns.tolist()
        valid_target_columns = valid_df.loc[:, valid_df.columns.str.startswith('d_')].columns.tolist()

        if not all([c in valid_df.columns for c in id_columns]):
            valid_df = pd.concat([train_df[id_columns], valid_df], axis=1, sort=False)

        self.train_df = train_df
        self.valid_df = valid_df
        self.calendar = calendar
        self.prices = prices

        self.weight_columns = weight_columns
        self.id_columns = id_columns
        self.valid_target_columns = valid_target_columns

        weight_df = self.get_weight_df()

        self.group_ids = (
            'all_id',
            'cat_id',
            'state_id',
            'dept_id',
            'store_id',
            'item_id',
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
        )

        for i, group_id in enumerate(tqdm(self.group_ids)):
            train_y = train_df.groupby(group_id)[train_target_columns].sum()
            scale = []
            for _, row in train_y.iterrows():
                series = row.values[np.argmax(row.values != 0):]
                scale.append(((series[1:] - series[:-1]) ** 2).mean())
            setattr(self, f'lv{i + 1}_scale', np.array(scale))
            setattr(self, f'lv{i + 1}_train_df', train_y)
            setattr(self, f'lv{i + 1}_valid_df', valid_df.groupby(group_id)[valid_target_columns].sum())

            lv_weight = weight_df.groupby(group_id)[weight_columns].sum().sum(axis=1)
            setattr(self, f'lv{i + 1}_weight', lv_weight / lv_weight.sum())

    def get_weight_df(self) -> pd.DataFrame:
        day_to_week = self.calendar.set_index('d')['wm_yr_wk'].to_dict()
        weight_df = self.train_df[['item_id', 'store_id'] + self.weight_columns].set_index(['item_id', 'store_id'])
        weight_df = weight_df.stack().reset_index().rename(columns={'level_2': 'd', 0: 'value'})
        weight_df['wm_yr_wk'] = weight_df['d'].map(day_to_week)

        weight_df = weight_df.merge(self.prices, how='left', on=['item_id', 'store_id', 'wm_yr_wk'])
        weight_df['value'] = weight_df['value'] * weight_df['sell_price']
        weight_df = weight_df.set_index(['item_id', 'store_id', 'd']).unstack(level=2)['value']
        weight_df = weight_df.loc[zip(self.train_df.item_id, self.train_df.store_id), :].reset_index(drop=True)
        weight_df = pd.concat([self.train_df[self.id_columns], weight_df], axis=1, sort=False)
        return weight_df

    def rmsse(self, valid_preds: pd.DataFrame, lv: int) -> pd.Series:
        valid_y = getattr(self, f'lv{lv}_valid_df')
        score = ((valid_y - valid_preds) ** 2).mean(axis=1)
        scale = getattr(self, f'lv{lv}_scale')
        return (score / scale).map(np.sqrt)

    def score(self, valid_preds: Union[pd.DataFrame, np.ndarray]) -> float:
        assert self.valid_df[self.valid_target_columns].shape == valid_preds.shape

        if isinstance(valid_preds, np.ndarray):
            valid_preds = pd.DataFrame(valid_preds, columns=self.valid_target_columns)

        valid_preds = pd.concat([self.valid_df[self.id_columns], valid_preds], axis=1, sort=False)

        group_ids = []
        all_scores = []
        for i, group_id in enumerate(self.group_ids):
            lv_scores = self.rmsse(valid_preds.groupby(group_id)[self.valid_target_columns].sum(), i + 1)
            weight = getattr(self, f'lv{i + 1}_weight')
            lv_scores = pd.concat([weight, lv_scores], axis=1, sort=False).prod(axis=1)
            group_ids.append(group_id)
            all_scores.append(lv_scores.sum())

        return group_ids, all_scores


In [7]:
df_lb = pd.read_csv(path/"m5-forecasting-accuracy-publicleaderboard-rank.csv")

## public LB rank
def get_lb_rank(score):
    """
    Get rank on public LB as of 2020-05-31 23:59:59
    """

    return (df_lb.Score <= score).sum() + 1


In [8]:
df_lb[df_lb['TeamName'].str.contains('gil') == True]

,TeamId,TeamName,SubmissionDate,Score,Rank
497,4713889,gil fernandes,2020-05-22 11:50:09,0.46521,498
3925,4543378,BjarnþórEgilsson,2020-03-21 16:03:04,1.08216,3926


In [9]:
## reading data
df_calendar = pd.read_csv(path/"calendar.csv")
df_prices = pd.read_csv(path/"sell_prices.csv")
df_sample_submission = pd.read_csv(path/"sample_submission.csv")
df_sample_submission["order"] = range(df_sample_submission.shape[0])

df_train = df_train_full.iloc[:, :-28]
df_valid = df_train_full.iloc[:, -28:]

evaluator = WRMSSEEvaluator(df_train, df_valid, df_calendar, df_prices)


In [10]:
## structure of validation data
preds_valid = df_valid.copy() + np.random.randint(100, size = df_valid.shape)
preds_valid.head()

,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,2,16,57,69,87,66,45,70,28,5,...,79,95,25,64,31,63,6,41,30,84
1,63,76,97,2,31,31,78,30,72,5,...,68,68,65,47,44,99,93,18,27,68
2,17,36,3,58,97,83,8,21,30,12,...,60,10,80,11,91,11,58,101,87,61
3,44,86,18,40,71,88,83,73,11,46,...,71,63,92,69,99,12,88,84,34,50
4,55,99,17,28,95,28,22,74,63,59,...,29,45,86,20,49,8,72,32,66,15


In [11]:
## evaluating random submission
groups, scores = evaluator.score(preds_valid)

score_public_lb = np.mean(scores)
score_public_rank = get_lb_rank(score_public_lb)

for i in range(len(groups)):
    print(f"Score for group {groups[i]}: {round(scores[i], 5)}")

print(f"\nPublic LB Score: {round(score_public_lb, 5)}")
print(f"Public LB Rank: {score_public_rank}")


Score for group all_id: 254.68728
Score for group cat_id: 265.96967
Score for group state_id: 232.56385
Score for group dept_id: 265.96827
Score for group store_id: 221.66515
Score for group item_id: 82.29932
Score for group ['state_id', 'cat_id']: 236.30871
Score for group ['state_id', 'dept_id']: 231.95546
Score for group ['store_id', 'cat_id']: 212.67252
Score for group ['store_id', 'dept_id']: 198.60304
Score for group ['item_id', 'state_id']: 51.61941
Score for group ['item_id', 'store_id']: 31.27034

Public LB Score: 190.46525
Public LB Rank: 4539


In [12]:
## evaluating submission from public kernel M5 - Three shades of Dark: Darker magic
## from https://www.kaggle.com/kyakovlev/m5-three-shades-of-dark-darker-magic
preds_valid = pd.read_csv("submission.csv")
preds_valid = preds_valid[preds_valid.id.str.contains("validation")]
preds_valid = preds_valid.merge(df_sample_submission[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
preds_valid.rename(columns = {
    "F1": "d_1914", "F2": "d_1915", "F3": "d_1916", "F4": "d_1917", "F5": "d_1918", "F6": "d_1919", "F7": "d_1920",
    "F8": "d_1921", "F9": "d_1922", "F10": "d_1923", "F11": "d_1924", "F12": "d_1925", "F13": "d_1926", "F14": "d_1927",
    "F15": "d_1928", "F16": "d_1929", "F17": "d_1930", "F18": "d_1931", "F19": "d_1932", "F20": "d_1933", "F21": "d_1934",
    "F22": "d_1935", "F23": "d_1936", "F24": "d_1937", "F25": "d_1938", "F26": "d_1939", "F27": "d_1940", "F28": "d_1941"
}, inplace = True)

groups, scores = evaluator.score(preds_valid)

score_public_lb = np.mean(scores)
score_public_rank = get_lb_rank(score_public_lb)

for i in range(len(groups)):
    print(f"Score for group {groups[i]}: {round(scores[i], 5)}")

print(f"\nPublic LB Score: {round(score_public_lb, 5)}")
print(f"Public LB Rank: {score_public_rank}")


Score for group all_id: 0.18807
Score for group cat_id: 0.25191
Score for group state_id: 0.31361
Score for group dept_id: 0.35103
Score for group store_id: 0.42105
Score for group item_id: 0.80556
Score for group ['state_id', 'cat_id']: 0.37565
Score for group ['state_id', 'dept_id']: 0.46195
Score for group ['store_id', 'cat_id']: 0.48725
Score for group ['store_id', 'dept_id']: 0.57249
Score for group ['item_id', 'state_id']: 0.8203
Score for group ['item_id', 'store_id']: 0.82592

Public LB Score: 0.48957
Public LB Rank: 1544
